## Writeup Template

### You can use this file as a template for your writeup if you want to submit it as a markdown file, but feel free to use some other method and submit a pdf if you prefer.

----

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./undistorted_chess.jpg "Undistorted chess"
[image2]: ./undistorted_test.jpg "Road Transformed"
[image3]: ./thresholding.jpg "Binary Example"
[image4]: ./tranform_straight.jpg "Warp Example"
[image5]: ./color_lines.jpg "Fit Visual"
[image6]: ./result2.jpg "Output"
[video1]: ./project_video.mp4 "Video"

## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

---

### Writeup / README

#### 1. Provide a Writeup / README that includes all the rubric points and how you addressed each one.  You can submit your writeup as markdown or pdf.  [Here](https://github.com/udacity/CarND-Advanced-Lane-Lines/blob/master/writeup_template.md) is a template writeup for this project you can use as a guide and a starting point.  

You're reading it!

### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the Camera Calibration code cell of the IPython notebook provided. 

I start by preparing "object points", which will be the (x, y, z) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on the (x, y) plane at z=0, such that the object points are the same for each calibration image.  Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection. 

In our case, we have choosen (9X6) corners chess, but 3/20 images provided fails to detect desired corners. So, we proceeded with other 17.

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result (notable the lines are straight now): 

![alt text][image1]

### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

To demonstrate this step, I will describe how I apply the distortion correction to one of the test images like this one:
In the 'Undistort' code cell, I have mentioned the code to undistort a test image provided in the example.
![alt text][image2]

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

I used a combination of color and gradient thresholds to generate a binary image (thresholding functions are defined in 'Color and Gradient Thresholding' code cell and used in pipeline function). I used 'x' gradient sobel and 's' color channel thresholding in HLS color space. Here's an example of my output for this step.  (note: this is not actually from one of the test images)

![alt text][image3]

#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.  

I have used cv2.getPerspectiveTransform() and cv2.warpPerspective() function for perspective transformation in the pipeline function. cv2.getPerspectiveTransform() takes 'src' and 'dst' as inputs. I chose the hardcode the source and destination points in the following manner:

This resulted in the following source and destination points:

| Source        | Destination   | 
|:-------------:|:-------------:| 
| 285, 670      | 250, 700      | 
| 1017, 670     | 950, 700      |
| 683, 450      | 950, 0        |
| 596, 450      | 250, 0        |

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image.

![alt text][image4]

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

Then I used sliding window approach (as written in sliding_window() function). This takes binary image and produces lane lines based on histogram and nonzero pixels in the expected region. Once the base of left and right lane is known in a frame, we can use it in next frame as a reference from where to start the search (explained in "if first =='True'" section of sliding_window function. 

![alt text][image5]

#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

I did this in 'Printing Radius of Curvature' code cell using the function print_roc() which takes image, left_fit and right_fit as input. Here, left-fit and right_fit are the coefficient of quadratic equation used to fit the left and right lane respectively.

Also, used avg of these coeffient over last 10 frames to avoid flatuation.

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I then fill the region between the two lanes with using cv2.fillpoly() in the sliding_window() funtion. Then took an inverse perspective tranform using Minv and weighted sum using cv2.addWeighted() function in the last few lines of pipeline function. Provided with a resultant image. 
![alt text][image6]

----

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Provided 'project_video_output.mp4' 

----

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?

Here I'll talk about the approach I took, what techniques I used, what worked and why, where the pipeline might fail and how I might improve it if I were going to pursue this project further.  

To detect lane lines in the image, tried different thresholding using color channel and gradient along x and y direction. As, few frames had dark yellow or shadows in it, so, 's' channel thresolding along with gradient along 'x' axis worked for me better. Have used average of 10 frames to calculate the curve of lane, that will avoid the flatuation in frames even if 2-3 frames' curve line are not detected properly. But, if more than 10 such frames are found, we will run into trouble (that is unlikely to happen and will have to go searching using sliding window approach from beginning). 

For improvement, we can have sanity checks in between to check if detected left and right lanes are parallel in bird-eye view and take necessary actions afterwards.

